In [1]:
import sqlite3

class GoogleNewsDBManager:
    def __init__(self):
        print ("DB Manager 시작")
        self.DBName = 'opinion_outside.db'
        self.db = sqlite3.connect(self.DBName, check_same_thread=False)
        self.db.row_factory = sqlite3.Row
        self.google_news_table = 'google_news'
        self.keyword_table = 'keyword'
        self.google_news_columns = {
            'published': 'text',
            'source': 'text PRIMARY KEY',
            'title': 'text',
            'link': 'text',
        }
        self.keyword_columns = {
            'keyword': 'text PRIMARY KEY',
            'country': 'text',
        }

    def __del__(self):
        self.stop()

    def stop(self):
        try: self.db.close()
        except: pass
    
    def queryCreateGoogleNewsTable(self, keyword):
        self.google_news_table = 'google_news_' + keyword.lower()
        cursor = self.db.cursor()
        colum_info = ",".join(col_name + ' ' + col_type for col_name, col_type in self.google_news_columns.items())
        query = "CREATE TABLE IF NOT EXISTS {} ({})".format(self.google_news_table, colum_info)
        cursor.execute(query)
        self.db.commit()

    def queryInsertGoogleNewsTable(self, values):
        cursor = self.db.cursor()
        colums = ','.join(self.google_news_columns.keys())
        values = '","'.join(str(values[col_name]).replace('"',"'") for col_name in self.google_news_columns.keys())
        query = 'INSERT OR IGNORE INTO {} ({}) VALUES ("{}")'.format(self.google_news_table, colums, values)
        cursor.execute(query)
        self.db.commit()

    def queryDeleteAllGoogleNewsTable(self, keyword):
        google_news_table = 'google_news_' + keyword.lower()
        query = "DROP TABLE IF EXISTS {}".format(google_news_table)
        cursor = self.db.cursor()
        cursor.execute(query)
        self.db.commit()

    def querySelectAllGoogleNewsTable(self, keyword):
        google_news_table = 'google_news_' + keyword.lower()
        query = "SELECT * FROM {}".format(google_news_table)
        #cursor = self.db.cursor()
        #result = cursor.execute(query)
        result = pd.read_sql(query,self.db)
        return result

    def queryCreateKeywordTable(self):
        cursor = self.db.cursor()
        colum_info = ",".join(col_name + ' ' + col_type for col_name, col_type in self.keyword_columns.items())
        query = "CREATE TABLE IF NOT EXISTS {} ({})".format(self.keyword_table, colum_info)
        cursor.execute(query)
        self.db.commit()

    def queryInsertKeywordTable(self, values):
        cursor = self.db.cursor()
        colums = ','.join(self.keyword_columns.keys())
        values = '","'.join(str(values[col_name]).replace('"',"'") for col_name in self.keyword_columns.keys())
        query = 'INSERT OR IGNORE INTO {} ({}) VALUES ("{}")'.format(self.keyword_table, colums, values)
        cursor.execute(query)
        self.db.commit()

    def queryDeleteKeywordTable(self, keyword):
        cursor = self.db.cursor()
        query = "DELETE FROM {} WHERE KEYWORD='{}'".format(self.keyword_table, keyword)
        cursor.execute(query)
        self.db.commit()

    def querySelectAllKeywordTable(self):
        query = "SELECT * FROM {}".format(self.keyword_table)
        cursor = self.db.cursor()
        cursor.execute(query)
        return cursor.fetchall()

In [2]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.base import JobLookupError
import requests
import datetime
#import maya
import feedparser
import pandas as pd


In [3]:
class GoogleNewsCron():
    def __init__(self):
        print ('크론 시작')
        self.scheduler = BackgroundScheduler(job_defaults={'max_instances': 10, 'coalesce': False})
        self.scheduler.start()
        self.dbManager = GoogleNewsDBManager()

    def __del__(self): 
        self.stop()

    def exec(self, country, keyword):
        print ('Google News Cron Start: ' + datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
        URL = 'https://news.google.com/rss/search?q={}+when:1d'.format(keyword)
        if country == 'en':
            URL += '&hl=en-NG&gl=NG&ceid=NG:en'
        elif country == 'ko':
            URL += '&hl=ko&gl=KR&ceid=KR:ko'

        try: 
            res = requests.get(URL)
            if res.status_code == 200:
                datas = feedparser.parse(res.text).entries
                for data in datas:
                    data['published'] = data.published
#                    data['published'] = '20220811'
#                    data['published'] = maya.parse(data.published).datetime(to_timezone="Asia/Seoul", naive=True) 
                    data['source'] = data.source.title
                    self.dbManager.queryInsertGoogleNewsTable(data)
            else:
                print ('Google 검색 에러')
        except requests.exceptions.RequestException as err:
            print ('Error Requests: {}'.format(err))
    
    def run(self, mode, country, keyword):
        print ("실행!")
        self.dbManager.queryCreateGoogleNewsTable(keyword)
        self.dbManager.queryCreateKeywordTable()
        self.dbManager.queryInsertKeywordTable({
            'keyword': keyword,
            'country': country
        })
        if mode == 'once':
            self.scheduler.add_job(self.exec, args=[country, keyword])
        elif mode == 'interval':
            self.scheduler.add_job(self.exec, 'interval', seconds=10, args=[country, keyword])
        elif mode == 'cron':
            self.scheduler.add_job(self.exec, 'cron', minute='*/10', args=[country, keyword])

    def stop(self):
        try: self.scheduler.shutdown() 
        except: pass
        try: self.dbManager.close() 
        except: pass

In [ ]:
import time

try:
    gooleNewsCron = GoogleNewsCron()
    # interval: 10초마다 수집(중복된 기사 제거)
    gooleNewsCron.run('cron', 'ko', '미래에셋증권')
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    gooleNewsCron.stop()


크론 시작
DB Manager 시작
실행!


/home/porter/anaconda3/lib/python3.9/site-packages/apscheduler/util.py:436: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return tzinfo.localize(dt)


Google News Cron Start: 08/17/2022, 22:50:00
Google News Cron Start: 08/17/2022, 23:00:00
Google News Cron Start: 08/17/2022, 23:10:00
Google News Cron Start: 08/17/2022, 23:20:00
Google News Cron Start: 08/17/2022, 23:30:00
Google News Cron Start: 08/17/2022, 23:40:00
Google News Cron Start: 08/17/2022, 23:50:00
Google News Cron Start: 08/18/2022, 00:00:00
Google News Cron Start: 08/18/2022, 00:10:00
Google News Cron Start: 08/18/2022, 00:20:00
Google News Cron Start: 08/18/2022, 00:30:00
Google News Cron Start: 08/18/2022, 00:40:00
Google News Cron Start: 08/18/2022, 00:50:00


### 테이블지우기

In [6]:
GoogleNewsDBManager().queryDeleteAllGoogleNewsTable("미래에셋증권")

DB Manager 시작


### 테이블 부르기

In [5]:
rows = GoogleNewsDBManager().querySelectAllGoogleNewsTable("미래에셋증권")

DB Manager 시작


In [6]:
rows

,published,source,title,link
0,"Wed, 10 Aug 2022 21:01:00 GMT",뉴스1,"미래에셋증권, 어닝서프라이즈에도 '목표가 하향' 왜? - 뉴스1",https://www.news1.kr/articles/?4769377
1,"Thu, 11 Aug 2022 00:09:00 GMT",매일경제,미래에셋증권(주) 상장지수증권 괴리율 초과 발생(미래에셋 Q150 Core5 ETN...,http://vip.mk.co.kr/news/view/21/28/1012669.html
2,"Thu, 11 Aug 2022 01:54:36 GMT",파이낸스 투데이,[빅데이터투데이] 증권사 2022년 8월 브랜드평판 랭킹... 1위 미래에셋증권 -...,http://www.fntoday.co.kr/news/articleView.html...
3,"Wed, 10 Aug 2022 08:57:29 GMT",CEOSCOREDAILY,"'1위 탈환' 미래에셋證, 해외 사업으로 '리딩 증권사' 굳히기 돌입 - CEOSC...",http://www.ceoscoredaily.com/page/view/2022081...
4,"Thu, 11 Aug 2022 05:21:49 GMT",더리포트,미래에셋증권 '신세계는 안전한 투자처' < 증권‧금융 < IT경제 < 기사본문 - ...,http://www.thereport.co.kr/news/articleView.ht...
...,...,...,...,...
63,"Thu, 11 Aug 2022 08:34:30 GMT",한국일보,코스피 상장 '쏘카'... 일반 청약 경쟁률 14대 1 - 한국일보,https://m.hankookilbo.com/News/Read/A202208111...
64,"Thu, 11 Aug 2022 08:40:02 GMT",비즈니스워치,"'하반기 대어' 쏘카, 일반청약도 흥행 실패...'경쟁률 14대 1' - 비즈니스워치",http://news.bizwatch.co.kr/article/market/2022...
65,"Thu, 11 Aug 2022 08:38:23 GMT",쿠키뉴스,"'IPO 대어' 쏘카, 공모가 낮춰 데뷔했지만 흥행 실패 - 쿠키뉴스",https://www.kukinews.com/newsView/kuk202208110168
66,"Wed, 10 Aug 2022 09:22:37 GMT",전국매일신문,대성하이텍 청약경쟁률 1136대 1 흥행...쏘카 첫날 3대 1 '부진' - 전국매일신문,https://www.jeonmae.co.kr/news/articleView.htm...


In [ ]:
df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
